<a href="https://colab.research.google.com/github/poseidon1412/Efficient-Customer-Review-Analysis-NLP-Project/blob/main/Project_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Datasets

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_yelp = pd.read_csv('/content/gdrive/MyDrive/nlp project/datasets/yelp_labelled.txt', sep='\t',header=None)

In [ ]:
data_yelp.head()
# review and sentiment
# 0-Negative, 1-Positive for positive review

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [ ]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
data_yelp.shape
# 1000 rows (reviews), 2 columns (Sentiments)

(1000, 2)

In [ ]:
data_amazon = pd.read_csv('/content/gdrive/MyDrive/nlp project/datasets/amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()
# review and sentiment
# 0-Negative, 1-Positive for positive review

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
data_amazon.columns = columan_name

In [ ]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
data_amazon.shape

(1000, 2)

In [ ]:
data_imdb = pd.read_csv('/content/gdrive/MyDrive/nlp project/datasets/imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
data_imdb.columns = columan_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
# Append all the data in a single dataframe
data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)

<ipython-input-41-126c26eba69a>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)


In [ ]:
data.shape

(2748, 2)

In [ ]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# check distribution of sentiments
data['Sentiment'].value_counts()

# 1346 positive reviews
# 1362 Negative reviews

1    1386
0    1362
Name: Sentiment, dtype: int64

In [ ]:
# check for null values
data.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

In [ ]:
x = data['Review']
y = data['Sentiment']

## Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
# Using the TextVectorization layer to normalize, split, and map strings
# to integers.
encoder = tf.keras.layers.TextVectorization(max_tokens=10000)
x = tf.convert_to_tensor(x.to_numpy(), dtype=tf.string)
# Adapt the encoder to the training data.
encoder.adapt(x)

# Create the model.
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model.
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Train the model on the training data.
history = model.fit(
    x=x,
    y=y,
    epochs=5
)

Epoch 1/5
86/86 [==============================] - 44s 215ms/step - loss: 0.6233 - accuracy: 0.5928
Epoch 2/5
86/86 [==============================] - 19s 218ms/step - loss: 0.2636 - accuracy: 0.9134
Epoch 3/5
86/86 [==============================] - 17s 201ms/step - loss: 0.1144 - accuracy: 0.9647
Epoch 4/5
86/86 [==============================] - 18s 205ms/step - loss: 0.0602 - accuracy: 0.9829
Epoch 5/5
86/86 [==============================] - 18s 216ms/step - loss: 0.0414 - accuracy: 0.9876


In [ ]:
sample_text = (
    '''The product quality is consistently outstanding, exceeding my expectations every time'''
)
predictions = model.predict(np.array([sample_text]))
print(*predictions[0])

# Print the label based on the prediction
if predictions[0] > 0:
    print('The review is positive')
else:
    print('The review is negative')

1/1 [==============================] - 6s 6s/step
10.16279
The review is positive


In [ ]:
# import pickle

# # save the iris classification model as a pickle file
# model_pkl_file = "project.pkl"

# with open(model_pkl_file, 'wb') as file:
#     pickle.dump(model, file)

In [ ]:
model.save('/content/gdrive/MyDrive/nlp project/finale_model', save_format='tf')
# model.save_weights("foobar_tf",save_format='tf')

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/gdrive/MyDrive/nlp project/finale_model')


In [ ]:
sample_text = (
    '''The product quality is consistently outstanding, exceeding my expectations every time'''
)
predictions = loaded_model.predict(np.array([sample_text]))
print(*predictions[0])

# Print the label based on the prediction
if predictions[0] > 0:
    print('The review is positive')
else:
    print('The review is negative')

1/1 [==============================] - 7s 7s/step
10.16279
The review is positive
